# Comparing Algorithms on FBST Dataset

In [ ]:
import sys
import random

# add library to path (or else, src not visible)
sys.path.insert(0, "../../")

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm

from src.AllocationSolver import AllocationSolver
from src.dists import SymmetricDiscreteDistribution, Distribution, UniformDistribution, NormalDistribution
from src.plots import plot_against_algos, plot_against_alpha, plot_versus_alpha, map_algo_to_name, map_algo_to_color, synthetic_summary_plots
from src.monte_carlo import hoeffding_bound
from src.format_data import clean_data

# use seaborn style plotting for poster presentation
sns.set_context("talk")
sns.set_style("whitegrid")

## Mobile Food Pantries

We have 70 demand locations, model with Normal demands.

In [ ]:
data = clean_data(pd.read_excel("../../data/mfp_food_pantries.xlsx"))

# create demand distributions
demand_dists = [NormalDistribution(mu, sigma) for mu, sigma in zip(data["avg_demand"], data["std_demand"])]
n = hoeffding_bound(0.01, 0.01)

In [ ]:
prob = AllocationSolver(
    demand_distributions=demand_dists,
    initial_supply=sum(demand_dists[i].mean() for i in range(len(demand_dists))),
    alpha=1,
    allocation_method="saffe"
)

In [ ]:
prob.change_allocation_method('ppa')
prob.monte_carlo_performance_metrics_batched(1000)

### Expected Waste vs. $\alpha$

In [ ]:
random.seed(1)
np.random.seed(1)

n = hoeffding_bound(0.1, 0.01)

def generator(alpha):
    prob.change_alpha(alpha)
    prob.change_allocation_method("lambda_time")
    prob.optimize_lambda(tol=0.1, monte_carlo=True, batch=True)
    return prob.monte_carlo_performance_metrics_batched(n)['waste']

plot_versus_alpha(generator, x_lab=rf"$\alpha$", y_lab="Expected Waste", title=rf"Expected Waste vs. $\alpha$")

algos = ["ppa", "greedy", "saffe"]
wastes = {
    "ppa": 108,
    "greedy": 92,
    "saffe": 131
}

for algo in algos:
    prob.change_allocation_method(algo)
    avg_waste = wastes[algo]
    plt.axhline(xmax=0, xmin=10, y=avg_waste, label=map_algo_to_name[algo], color=map_algo_to_color[algo])

plt.legend()